# Linear and Nonlinear Regression
## Eric Feigelson 
## Summer School in Statistics for Astronomers

**In this tutorial, we exercise two types of regression using R and its CRAN packages.  First, we consider cases where a global mathematical function is assumed to apply to the full dataset.  These include linear and nonlinear regression with options such as weighting my measurement errors and robust treatment of outliers.  A crucial aspect is goodness-of-fit and residual analysis to confirm that the best-fit model is indeed a good fit.  Second we consider several local regression methods where a continuous curve is estimated from sequential pieces of the data.**

## 1. Linear modeling

We start by running one of the most widely used functions in R, _lm_ for linear modeling, and related procedures. But vefore we start, it is important to note that, in statistical parlance, 'linear' refers to a much broader range of models than $Y = \beta_0 + \beta_1 X + \epsilon$.  It includes any model that is linear in the model **parameters**.  Thus the following models are 'linear' and an encompassed by the powerful theorems underlying linear modeling:
- $Y = \beta_0 + \beta_1 X + \beta_2 X^2 + \epsilon$ (polynomial)
- $Y = \beta_0 e^{-X} + \epsilon$ (exponential)
- $Y = \beta_0 + \beta_1 sin X + \beta_2 cos X + \epsilon$ (sinusoid)

In contrast the following models are nonlinear:
- $Y = (X/\beta_0)^{-\beta_1}+ \epsilon$  power law (Pareto)
- $Y = \beta_0 e^{-X/\beta_1} + \epsilon$ (scaled exponential)
- $Y = \beta_0 / (1 + (X/\beta_1)^2) + \epsilon$  isothermal sphere
- $Y = \beta_0 + \beta_1 sin(X + \beta_2) + \beta_3 cos(X + \beta_2) + \epsilon$  sinusoid with arbitrary phase
- $Y = [\beta_0 + \beta_1 X for X < \beta_2] and [\beta_3 + \beta_4 X for X > \beta_2]$ segmented linear
- Most astrophysical models like $\Lambda$CDM cosmology 

We will practice linear modeling with a collection of photometry of spectroscopically confirmed quasars from the Sloan Digital Sky Sruvey.  We examine a relationship between the magnitdues in two bands; this is scientifically rather useless, but gives opportunity to test methodology for simple linear regression with difficulties common in astronomical regressions:  non-Gaussian scatter, heteroscedastic measurement errors, and outliers.  

Note that the theory of linear modeling is not restricted to bivariate problems: it is intrinsically multivariate in the sense that a single response variable $Y$ can be a function of a vector of covariates $\bf{X}$ as in the simple model: $Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p + \epsilon$.

In [0]:
# I. Construct large and small samples of 77K SDSS quasars

if(!require("astrodatR", quietly=T)) {
  install.packages("astrodatR", repos="https://cloud.r-project.org", dependencies=TRUE)
}; library("astrodatR")
data(SDSS_QSO)     # brings tabular data into an R data.frame

# Basic information about a data.frame
dim(SDSS_QSO)      # number of rows and columns
names(SDSS_QSO)    # names of columns
summary(SDSS_QSO)  # 5-number summary of each column

In [0]:
# Remove some bad photometry

qso <- SDSS_QSO[-which(SDSS_QSO[,3] == 0 | SDSS_QSO[,9] == 0),]
qso <- qso[-which(qso[,4] == 9.999 | qso[,12] == 9.999),]
qso[(qso[,4]<0.02),4 ] <- 0.02        # set threshold on magnitude errors
dim(qso) ; summary(qso)
attach(qso)

When applied to a data.frame, the R function `attach` allows the user to access a column by its names (e.g. r_mag) without remembering their column number (e.g. qso[,3]).

In [0]:
# Plot dataset of SDSS quasar i vs. u magnitudes showing 
# heteroscedastic measurement errors, with contours for dense regions

options(jupyter.plot_scale=1)
options(repr.plot.width = 7, repr.plot.height = 5)

plot(i_mag, u_mag, pch=20, cex=0.1, col='#00000040', xlim=c(16,21), 
   ylim=c(16.5,23.5), xlab="SDSS i (mag)", ylab="SDSS u (mag)")
for(i in 50:150) {
   lines(c(i_mag[i],i_mag[i]),c((u_mag[i]+sig_u_mag[i]),
      (u_mag[i]-sig_u_mag[i])), lwd=2, col='purple2')
   lines(c((i_mag[i]+sig_i_mag[i]),(i_mag[i]-sig_i_mag[i])),
      c(u_mag[i],u_mag[i]), lwd=2, col='purple2')   }

if(!require("KernSmooth", quietly=T)) {
  install.packages("KernSmooth", repos="https://cloud.r-project.org", dependencies=TRUE)
}; library("KernSmooth")
smqso <- bkde2D(cbind(i_mag, u_mag), bandwidth=c(0.05, 0.05), gridsize=c(400,400))
contour(smqso$x1, smqso$x2, smqso$fhat, add=T, col='gold', nlevels=9)

Here we see a disturbed dataset. There is wide asymmetrical scatter towards main magnitudes in the SDSS u (ultraviolet) band.  Much of the scatter is attributable to measurement errors, but not all of it. Let us now run _lm_ with the formula: $u = \beta_0 + \beta_1 i + \epsilon$, obtain 90% confidence interval of the intercept and slope parameters, and show the scatter plot with the fitted line. 

In [0]:
# II. Ordinary least squares fit

fit_ols <- lm(u_mag~i_mag)
summary(fit_ols) 
confint(fit_ols, level=0.90)          # 3 sigma equivalent for Gaussian distribution

plot(i_mag, u_mag, pch=20, cex=0.1, col='#00000040', xlim=c(16,21), 
   ylim=c(16.5,23.5), xlab="SDSS i (mag)", ylab="SDSS u (mag)")
abline(fit_ols$coef, lty=1, lwd=2)     # solid black line

Note that, since the scatter is non-Gaussian, the theorems underlying ordinary least squares means that the fit is not a maximum likelihood estimator and the parameter uncertainties may not be reliable.  Visually, this is a terrible fit, missing most of the data points.  We now try to improve it in three ways:  weighting by measurement errors; applying `robust` downweighting of outliers; and applying both corrections.  

In [0]:
# III. Weighted least squares fit

fit_wt <- lm(u_mag~i_mag, x=T, weights=1/(sig_u_mag*sig_u_mag))
summary(fit_wt)

plot(i_mag, u_mag, pch=20, cex=0.1, col='#00000040', xlim=c(16,21), 
   ylim=c(16.5,23.5), xlab="SDSS i (mag)", ylab="SDSS u (mag)")
abline(fit_wt$coef,lty=2,lwd=2, col='green')    # dashed green line

This is the ***minimum chi-squared regression*** commonly used by astronomers; statisticians would call it ***weighted least squares regression with weights from heteroscedastic measurement errors***.  Here we see that most of the problems can be removed with measurement error weighting. However, the calculation is obviously wrong in this case because (in statistical parlance) the model is misspecified because only a portion of the scatter is caused by measurement error.  

Astronomers often do not carefully examine the accuracy and validity of their regression fits.  Diagnostic graphics are very useful for this.  Here are the plots produced automatically by R's `lm` function. For interpretation and details, see the text [_A Modern Approach to Regression with R_](https://link.springer.com/book/10.1007%2F978-0-387-09608-7) (S. Sheather, 2009).

In [0]:
# Diagnostic plots involving regression residuals help identify outliers

par(mfrow=c(2,2))		
plot(fit_wt, which=c(2:5), caption='', sub.caption='' ,pch=20, cex=0.3, 
   cex.lab=1.3, cex.axis=1.3)

Other regression diagnostic tests can be applied to help evaluate the validity of the statistical model and its best-fit parameters.  The most important is a nonparametric two-sample test between the cumulative distribution functions of the observed and fitted values of the response variable. The Kolmogorov-Smirnov test is commonly used, but the Cramer-von Mises and Anderson-Darling tests are more sensitive.   However, theorems show that tabulated probabilities are inapplicable when the second distribution is from a model based on the first distribution.  We thus use bootstrap replications to estimate probabilities. This capability is provided by CRAN package 'cramer' for the Cramer-von Mises test.  To avoid excessive computational time, we only treat the first 1000 points here. The weighted linear fit is obviously rejected by the test.  

In [0]:
# Goodness-of-fit for a 2D regression

if(!require("cramer", quietly=T)) {
  install.packages("cramer", repos="https://cloud.r-project.org", dependencies=TRUE)
}; library("cramer")
cramer.test(fit_wt$model$u_mag[1:1000], fit_wt$fitted.values[1:1000], replicates=100)

plot(ecdf(fit_wt$model$u_mag[1:1000]), cex=0, col='royalblue', lwd=2, xlab='SDSS u (mag)', ylab='c.d.f.', main='Observed and fitted u magnitude')
plot(ecdf(fit_wt$fitted.values[1:1000]), cex=0, col='magenta', lwd=2, add=TRUE)
text(22, 0.5, 'Observed', cex=1.5, col='royalblue')
text(19, 0.8, 'Model', cex=1.5, col='magenta')

Another approach to non-Gaussianity and outliers is to apply _robust_ regression techniques.  These are many variants; here R's _rlm_ (robust linear modeling) function, downweighting outliers using Huber's psi function, with and without measurement error weighting.  Unfortunately, this code does not have a built-in line plotting option, so we draw the lines manually from information in the _rlm_ output.  See various approaches in R at the CRAN Task View on Robust Statistics.

In [0]:
# Robust M-estimator

library(MASS)
fit_M <- rlm(u_mag~i_mag, method='M')	# robust fit with Huber's psi functon
summary(fit_M)  
aM <- fit_M$coef[[1]] ; bM <- fit_M$coef[[2]]

plot(i_mag, u_mag, pch=20, cex=0.1, col='#00000040', xlim=c(16,21), 
   ylim=c(16.5,23.5), xlab="SDSS i (mag)", ylab="SDSS u (mag)")
lines(c(16,21), c(aM+bM*16, aM+bM*21), lty=3, lwd=3, col='royalblue3') # dotted royal blue line

fit_Mwt <- rlm(u_mag~i_mag, method='M', weights=1/(sig_u_mag*sig_u_mag), 
   wt.method='inv.var')   # robust fit with measurement error weighting 
summary(fit_Mwt)  
aMwt <- fit_Mwt$coef[[1]] ; bMwt <- fit_Mwt$coef[[2]]

lines(c(16,21), c(aMwt+bMwt*16, aMwt+bMwt*21), lty=3, lwd=5, col='orange')
text(19.5, 17, 'u = 0.13 + 1.02*i', cex=1.3, col='orange')

abline(fit_wt$coef,lty=2,lwd=2, col='green')    # dashed green line (weighted LS fit)

Here we see that a robust regression line that downweights outliers treated most of the problem with bad u-band photometry.  And a line that treats ***both outliers and measurement errors*** did a great job (orange line).  This is probably the preferred result for this problem.  But of course, the model is misspecified as the residuals are far from homoscedastic Gaussian noise, so the scientific meaning of the result (slope = 1.02) is suspect. 


> **Exercise 1**.  Try different robustification options in _rlm_ such as MM estimation and different choices of removal of outlying points (options in _lqs_). 

### 2. Nonlinear regression

<center><img src="Ilc_9yr_moll4096.png" width=400>  <img src="587px-PowerSpectrumExt.svg.png" width=400></center>

Astronomers often fit data with nonlinear functions derived from astrophysical theory that we believe apply to the observed situation. Among important recent applications of nonlinear regression are the fitting of the consensus Lambda-CDM model of cosmology to the fluctuations in the cosmic microwave background (images above, WMAP results) and the fitting of Keplerian exoplanetary orbits to stellar radial velocity time series.  

But astronomers also often fit data with heuristic nonlinear functions that do not have a clear astrophysical interpretation such as the stellar Initial Mass Function (several distributions used), galaxy luminosity function (Schechter function = gamma distribution), Navarro-Frenk-White Dark Matter profile, and various galaxy scaling relations.

Here we fit radial profiles from nearby Virgo Cluster elliptical galaxies to a heuristic nonlinear function proposed by Jose Luis Sersic in 1968. Here the surface brightness $B$ of an elliptical galaxy (or spiral galaxy bulge) as a function of radius $r$ follows: $B \propto -2.5*log(I_e * 10^{-(0.868*n-0.142)}((r/r_e)^{1/n}-1))$.  The data are obtained from [Kormendy et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJS..182..216K/abstract). We fit using R's _nls_ (nonlinear least squares) function;  see also CRAN package _nmle_ for maximum likelihood fitting.

In [0]:
# Unpack galaxy profile data

data(ell_gal_profile)
summary(ell_gal_profile)
NGC4472 <- ell_gal_profile[ell_gal_profile[,1] == 'NGC.4472',2:3]
NGC4472
radius <- NGC4472[,1]
surf_mag <- NGC4472[,2]

NGC4472.fit <-  nls(surf_mag ~ -2.5*log10(I.e * 10^(-(0.868*n-0.142)*
   ((radius/r.e)^{1/n}-1))) + 26, data=list(NGC4472), start=list(I.e=20.,
   r.e=120.,n=4.), model=T, trace=T)
summary(NGC4472.fit)
logLik(NGC4472.fit)


In [0]:
# Plot NGC 4472 data and best-fit model

par(mai=c(1,1,0.8,0.44))   # improve left-hand margin
plot(NGC4472.fit$model$radius, NGC4472.fit$model$surf_mag, pch=20, 
   xlab="r  (arcsec)", ylab=expression(mu ~~ (mag/sq.arcsec)), ylim=c(16,28), 
   cex.lab=1.5, cex.axis=1.5)
lines(NGC4472.fit$model$radius, fitted(NGC4472.fit), lw=2, col='red')

We can examine various scalar quantities from the _nls_ fit, and plot the residuals between the data and model.  A nonparametric smoother is added to assist seeing the amazing structure in the residuals: periodic shells of stars in excess of the monotonic Sersic model.  This is a well-known effect due to past galaxy mergers that form large elliptical galaxies. A similar residual plot appears in Kormendy's paper. 

In [0]:
# Details information about the nls fit

formula(NGC4472.fit)    # formula used
coef(NGC4472.fit)       # best-fit parameters
vcov(NGC4472.fit)       # best-fit parameter covariance matrix
logLik(NGC4472.fit)     # log-likelihood of best fit

In [0]:
confint(NGC4472.fit)    # 95% confidence intervals
profile(NGC4472.fit)    # profiles (cuts) around the best fit (unfortunately the console output is verbose here)

In [0]:
fitted(NGC4472.fit)     # fitted values
residuals(NGC4472.fit)  # residuals from the fitted values

# Residual plot

plot(NGC4472.fit$model$radius,residuals(NGC4472.fit), xlab="r (arcsec)", 
   ylab="Residuals", pch=20, cex.lab=1.5, cex.axis=1.5)
lines(supsmu(NGC4472.fit$model$radius, residuals(NGC4472.fit), span=0.05), 
   lwd=2, col='red')

We can perform more analysis of the residuals.  First, we show the residuals are normally distributed (Shapiro-Wilks test) but exhibit strong spatial autocorrelation (Durbin-Watson test). 

In [0]:
# Test for normality (OK) and autocorrelation (not OK) of residuals
# For linear models, also use the Durbin-Watson test in CRAN packages lmtest and car

qqnorm(residuals(NGC4472.fit) / summary(NGC4472.fit)$sigma) 
abline(a=0,b=1)
shapiro.test(residuals(NGC4472.fit) / summary(NGC4472.fit)$sigma) 
acf(residuals(NGC4472.fit))

There is an oddity: the error on Sersic's _n_ parameter from `nls` is much smaller than the error quoted by Kormendy.  Reading Kormendy's appendix, I find that he did not know how to evaluate the uncertainty of a nonlinear fit and chose an _ad hoc_ procedure that overestimated the error.  His estimate of _n_ was much more accurate than he thought.

> **Exercise 2.**  (a) See whether the best-fit model is significantly different using maximum likelihood estimation (CRAN package _nmle_) rather than Iteratively Weighted Least Squares (_nls_ in R).  (b) Estimate parameter confidence intervals using bootstrap techniques using CRAN _nlstools_.

### Some useful books for regression

- S. Sheather, [_A Modern Approach to Regression with R_](https://link.springer.com/book/10.1007%2F978-0-387-09608-7), 2009, Springer
- J. Fox, [_An R Companion to Applied Regression_](https://socialsciences.mcmaster.ca/jfox/Books/Companion/), 3rd ed, 2019, Sage 
- C. Ritz & J. Streibig, [_Nonlinear Regression with R_](https://link.springer.com/book/10.1007/978-0-387-09616-2), 2008, Springer
- K. Takezawa, [_Introduction to Nonparametric Regression_](https://onlinelibrary.wiley.com/doi/book/10.1002/0471771457), 2005, Wiley
- J. Harazlak, D. Ruppert & M. Wand, [_Semiparametric Regression with R_](https://link.springer.com/book/10.1007/978-1-4939-8853-2),  2018, Springer